# ODTC Door Command Test

This notebook demonstrates connecting to an ODTC device and testing the open/close door commands.

## Setup

Before running, ensure:
1. The ODTC device is powered on and connected to the network
2. You know the IP address of the ODTC device
3. Your computer is on the same network as the ODTC
4. The ODTC device is accessible (not locked by another client)

In [1]:
# Import required modules
import asyncio
import sys
from pathlib import Path

from pylabrobot.thermocycling.inheco import InhecoODTC, ODTCBackend

In [2]:
# Configuration
# TODO: Replace with your ODTC device IP address
ODTC_IP = "192.168.1.50"  # Change this to your ODTC's IP address

print(f"Will connect to ODTC at {ODTC_IP}")

Will connect to ODTC at 192.168.1.50


In [3]:
# Use model="96" for 96-well or model="384" for 384-well format
tc = InhecoODTC(name="odtc_test", backend=ODTCBackend(odtc_ip=ODTC_IP), model="96")

print("Backend and thermocycler objects created.")

Backend and thermocycler objects created.


## Connection Management

In [4]:
# Connect and initialize the device
# This performs the full SiLA connection lifecycle:
# 1. Sets up HTTP event receiver server
# 2. Calls Reset (Startup -> Standby) to register event receiver
# 3. Calls Initialize (Standby -> Idle) to ready the device
await tc.setup()
print("✓ Connected and initialized successfully!")
print("Device should now be in Idle state and ready for commands.")

2026-01-26 13:57:19,244 - pylabrobot.storage.inheco.scila.inheco_sila_interface - INFO - Device reset (unlocked)


✓ Connected and initialized successfully!
Device should now be in Idle state and ready for commands.


## Status Commands

In [5]:
# Get device status
# After setup(), device should be in "Idle" state
status = await tc.get_status()
print(f"Device status: {status}")

Device status: idle


In [6]:
# Get methods and premethods stored on the device
# Note: Requires device to be in Idle state (after setup/initialize)
# Returns ODTCMethodSet object with premethods and methods attributes
method_set = await tc.get_method_set()

print(f"PreMethods ({len(method_set.premethods)}):")
if method_set.premethods:
    for pm in method_set.premethods:
        print(f"  - {pm.name}")
else:
    print("  (none)")

print(f"\nMethods ({len(method_set.methods)}):")
if method_set.methods:
    for m in method_set.methods:
        print(f"  - {m.name}")
else:
    print("  (none)")

# You can also get a specific method by name:
# method = await tc.get_method_by_name("PCR_30cycles")

PreMethods (15):
  - PRE25
  - PRE25LID50
  - PRE55
  - PREDT
  - Pre_25
  - Pre25
  - Pre_25_test
  - Pre_60
  - Pre_4
  - dude
  - START
  - EVOPLUS_Init_4C
  - EVOPLUS_Init_110C
  - EVOPLUS_Init_Block20CLid85C
  - EVOPLUS_Init_Block20CLid40C

Methods (61):
  - M18_Abnahmetest
  - M23_LEAK
  - M24_LEAKCYCLE
  - M22_A-RUNIN
  - M22_B-RUNIN
  - M30_PCULOAD
  - M33_Abnahmetest
  - M34_Dauertest
  - M35_VCLE
  - M36_Verifikation
  - M37_BGI-Kon
  - M39_RMATEST
  - M18_Abnahmetest_384
  - M23_LEAK_384
  - M22_A-RUNIN_384
  - M22_B-RUNIN_384
  - M30_PCULOAD_384
  - M33_Abnahmetest_384
  - M34_Dauertest_384
  - M35_VCLE_384
  - M36_Verifikation_384
  - M37_BGI-Kon_384
  - M39_RMATEST_384
  - M120_OVT
  - M320_OVT
  - M121_OVT
  - M321_OVT
  - M123_OVT
  - M323_OVT
  - M124_OVT
  - M324_OVT
  - M125_OVT
  - M325_OVT
  - PMA cycle
  - Test
  - DC4_ProK_digestion
  - DC4_3Prime_Ligation
  - DC4_ProK_digestion_1_test
  - DC4_3Prime_Ligation_test
  - DC4_5Prime_Ligation_test
  - DC4_USER_Ligatio

### Non-Blocking Door Operations

The door commands support non-blocking execution using the `wait=False` parameter. This returns a `CommandExecution` handle that you can await later, allowing you to do other work while the door operation completes. 

Just set wait=True to treat these as blocking and run them as normal commands

In [7]:
# Non-blocking door close
print("Starting door close (non-blocking)...")
door_closing = await tc.close_door(wait=False)
print(f"✓ Door close command started! Request ID: {door_closing.request_id}")

Starting door close (non-blocking)...
✓ Door close command started! Request ID: 1783384016


In [8]:
# Access DataEvents for a command execution
# (Note: DataEvents are primarily used for method execution, but the pattern works for all commands)
# You can also use the wait() method explicitly
print("Waiting for door to close...")
await door_closing.wait()
print("✓ Door close completed!")

door_opening = await tc.open_door(wait=False)
print(f"Door opening with request ID: {door_opening.request_id}")

# Get DataEvents for this command (if any were collected)
events = await door_opening.get_data_events()
print(f"DataEvents collected: {len(events)}")

# Wait for completion
await door_opening
print("✓ Door open completed!")

Waiting for door to close...
✓ Door close completed!
Door opening with request ID: 1991913242
DataEvents collected: 0
✓ Door open completed!


# Close the connection

In [ ]:
# Close the connection
await tc.stop()
print("✓ Connection closed.")